In [6]:
from sklearn import preprocessing, model_selection, svm
import quandl, datetime
import pandas as pd
import math
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
%matplotlib qt 

In [7]:
style.use('ggplot')
df = quandl.get('WIKI/GOOGL')

In [8]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [9]:
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [10]:
df['HC_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

In [11]:
df = df[['Adj. Close','HC_PCT','PCT_change','Adj. Volume']]

In [12]:
df.head()

,Adj. Close,HC_PCT,PCT_change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [13]:
forecast_col = 'Adj. Close'

In [14]:
df.fillna(-99999, inplace=True)

In [15]:
forecast_out = int(math.ceil(0.01*len(df))) #math.ceil rounds everythik to the nearest number// trying to predict 10% of the data
print(forecast_out)

35


In [16]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [17]:
df.head()

,Adj. Close,HC_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.078238
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.839414
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,1.886792,1.183658,9188600.0,71.219849


In [18]:
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:] # stuff we gonna predict

In [19]:
X = X[:-forecast_out]

In [20]:
#X = X[:-forecast_out+1]
#df.dropna(inplace=True)
df.dropna(inplace=True)
y=np.array(df['label'])
y = np.array(df['label'])

In [21]:
len(X), len(y)

(3389, 3389)

In [38]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2) 
# clf = LinearRegression(n_jobs=-1)
# clf.fit(X_train, y_train)
# with open('linearregression.pickle','wb') as f:
#     pickle.dump(clf, f)
    
pickle_in = open('linearregression.pickle','rb')
clf = pickle.load(pickle_in)

accuracy = clf.score(X_test, y_test)

In [39]:
forecast_set = clf.predict(X_lately)
forecast_set

array([1103.9534442 , 1075.96593165, 1025.33768996, 1062.70063691,
       1074.8823581 , 1074.88151202, 1093.30556919, 1112.44434782,
       1116.50739913, 1124.67339682, 1134.75870841, 1131.43717289,
       1149.81600742, 1165.47481982, 1138.95535645, 1125.00346115,
       1092.05772138, 1104.56212912, 1115.82286672, 1122.45526258,
       1136.28583223, 1150.95353337, 1182.44327774, 1187.71375791,
       1161.51108082, 1170.5007239 , 1172.4139353 , 1155.8280507 ,
       1120.92565322, 1116.65385601, 1114.96495984, 1073.30879773,
       1046.59992398, 1074.22401135, 1026.79650891])

In [40]:
accuracy

0.9780008956846367

In [41]:
forecast_out

35

In [42]:
df['Forecast'] = np.nan

In [43]:
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day

In [44]:
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns) -1)] + [i]

In [45]:
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()